In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

from mapie.regression import MapieRegressor
from mapie.metrics import regression_coverage_score

import pickle
import time
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
traffic_df = pd.read_csv('Traffic_Volume.csv')
traffic_df

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,date_time,traffic_volume
0,NaN,288.28,0.0,0.0,40,Clouds,10/2/12 9:00,5545
1,NaN,289.36,0.0,0.0,75,Clouds,10/2/12 10:00,4516
2,NaN,289.58,0.0,0.0,90,Clouds,10/2/12 11:00,4767
3,NaN,290.13,0.0,0.0,90,Clouds,10/2/12 12:00,5026
4,NaN,291.14,0.0,0.0,75,Clouds,10/2/12 13:00,4918
...,...,...,...,...,...,...,...,...
48199,NaN,283.45,0.0,0.0,75,Clouds,9/30/18 19:00,3543
48200,NaN,282.76,0.0,0.0,90,Clouds,9/30/18 20:00,2781
48201,NaN,282.73,0.0,0.0,90,Thunderstorm,9/30/18 21:00,2159
48202,NaN,282.09,0.0,0.0,90,Clouds,9/30/18 22:00,1450


In [4]:
output = traffic_df['traffic_volume']
input = traffic_df.drop(columns = 'traffic_volume')
input.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,date_time
0,NaN,288.28,0.0,0.0,40,Clouds,10/2/12 9:00
1,NaN,289.36,0.0,0.0,75,Clouds,10/2/12 10:00
2,NaN,289.58,0.0,0.0,90,Clouds,10/2/12 11:00
3,NaN,290.13,0.0,0.0,90,Clouds,10/2/12 12:00
4,NaN,291.14,0.0,0.0,75,Clouds,10/2/12 13:00


In [5]:
input['date_time'] = pd.to_datetime(input['date_time'])
input['year'] = input['date_time'].dt.year
input['month'] = input['date_time'].dt.month_name()
input['day'] = input['date_time'].dt.day_name()
input['hour'] = input['date_time'].dt.hour
input = input.drop(columns = 'date_time')
input.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,year,month,day,hour
0,NaN,288.28,0.0,0.0,40,Clouds,2012,October,Tuesday,9
1,NaN,289.36,0.0,0.0,75,Clouds,2012,October,Tuesday,10
2,NaN,289.58,0.0,0.0,90,Clouds,2012,October,Tuesday,11
3,NaN,290.13,0.0,0.0,90,Clouds,2012,October,Tuesday,12
4,NaN,291.14,0.0,0.0,75,Clouds,2012,October,Tuesday,13


In [6]:
dummy_input = pd.get_dummies(input)
dummy_input.drop(columns = 'year', inplace = True)
dummy_input.head()

,temp,rain_1h,snow_1h,clouds_all,hour,holiday_Christmas Day,holiday_Columbus Day,holiday_Independence Day,holiday_Labor Day,holiday_Martin Luther King Jr Day,...,month_November,month_October,month_September,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
0,288.28,0.0,0.0,40,9,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
1,289.36,0.0,0.0,75,10,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
2,289.58,0.0,0.0,90,11,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3,290.13,0.0,0.0,90,12,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,291.14,0.0,0.0,75,13,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False


In [7]:
train_X, test_X, train_y, test_y = train_test_split(dummy_input, output, test_size = 0.2, random_state = 1)

In [8]:
xg_estimator = XGBRegressor(random_state = 1)

In [9]:
start = time.time()
xg_estimator.fit(train_X, train_y)
stop = time.time()
print(f'Time Taken: {stop - start} seconds')

Time Taken: 0.34093689918518066 seconds


In [10]:
y_pred = xg_estimator.predict(test_X)

r2 = sklearn.metrics.r2_score(test_y, y_pred)
r2

0.955457329750061

In [11]:
y_pred_train = xg_estimator.predict(train_X)
r2_train = sklearn.metrics.r2_score(train_y, y_pred_train)
r2_train


0.9643148183822632

In [12]:
residuals = test_y - y_pred

traffic_res_hist = plt.figure(figsize = (10, 5), dpi = 200)
plt.hist(residuals, bins=200, color = 'blue', edgecolor = 'black')

plt.xlabel('Residuals', fontsize = 14)
plt.ylabel('Frequency', fontsize = 14)
plt.title('Distribution of Residuals', fontsize = 18)
plt.savefig('xg_res_hist.svg')

In [13]:
importance = xg_estimator.feature_importances_

feat_imp = pd.DataFrame(list(zip(train_X.columns, importance)),
                        columns = ['Feature', 'Importance'])
feat_imp = feat_imp.sort_values(by = 'Importance', ascending = False).reset_index(drop=True).head(20)

xg_feat_imp = plt.figure(figsize=(6, 4))
plt.barh(feat_imp['Feature'], feat_imp['Importance'], color = 'blue', edgecolor = 'black')

plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance Bar Plot (Top 20)')
plt.tight_layout()
plt.savefig('xg_feature_importance.svg')

In [14]:
xg_scatter = plt.figure(figsize=(6, 4))

plt.scatter(test_y, y_pred, color = 'blue', alpha = 0.7, edgecolor = 'black', s = 40)

plt.plot([min(test_y), max(test_y)], [min(test_y), max(test_y)], color = 'red', linestyle = '--', lw = 2)

plt.xlabel('Actual Values', fontsize = 12)
plt.ylabel('Predicted Values', fontsize = 12)
plt.title('Predicted vs Actual Plot', fontsize = 16)
plt.tight_layout()
plt.savefig('xg_scatter.svg')


In [15]:
mapie = MapieRegressor(estimator=xg_estimator,
                       n_jobs = -1,
                       random_state = 1)

In [16]:
start = time.time()
mapie.fit(train_X, train_y)
stop = time.time()
print(f'Time Taken: {stop - start} seconds')

Time Taken: 5.193628311157227 seconds


In [17]:
alpha = 0.1
y_pred, y_pred_interval = mapie.predict(test_X, alpha=alpha)

In [18]:
y_pred

array([ 912.894 , 1207.1001, 1986.1038, ..., 1646.6497, 4603.28  ,
       4533.1685], dtype=float32)

In [19]:
y_pred_interval

array([[[ 299.54248047],
        [1442.3927002 ]],

       [[ 534.08312988],
        [1701.88952637]],

       [[1410.7322998 ],
        [2578.76806641]],

       ...,

       [[1069.2434082 ],
        [2215.41845703]],

       [[4073.69238281],
        [5224.30004883]],

       [[4127.58251953],
        [5278.77685547]]])

In [20]:
coverage = regression_coverage_score(test_y,
                                     y_pred_interval[:, 0],
                                     y_pred_interval[:, 1])

cov_perc = coverage * 100

In [21]:
mapie_df = test_y.to_frame()
mapie_df.columns = ['Actual Value']
mapie_df['Predicted Value'] = y_pred.round(2)
mapie_df['Lower Value'] = y_pred_interval[:, 0].round(2)
mapie_df['Upper Value'] = y_pred_interval[:, 1].round(2)
mapie_df.head()

,Actual Value,Predicted Value,Lower Value,Upper Value
6715,812,912.890015,299.54,1442.39
27836,1241,1207.099976,534.08,1701.89
43682,1703,1986.099976,1410.73,2578.77
41005,5829,5894.290039,5287.23,6432.28
28608,3798,3742.780029,3372.77,4536.13


In [22]:
sorted_mapie_df = mapie_df.sort_values(by = ['Actual Value']).reset_index(drop = True)

xg_coverage, ax = plt.subplots(figsize = (6, 4))

plt.plot(sorted_mapie_df['Actual Value'], 'blue', markersize=3, label='Actual Value')

plt.fill_between(np.arange(len(sorted_mapie_df)),
                 sorted_mapie_df['Lower Value'],
                 sorted_mapie_df['Upper Value'],
                 alpha=0.2, 
                 color='blue', 
                 label='Prediction Interval')

plt.xlim([0, len(sorted_mapie_df)])

plt.xlabel('Samples', fontsize=12)
plt.ylabel('Target', fontsize=12)

plt.title(f'Prediction Intervals and Coverage: {cov_perc:.2f}%', fontsize=14, fontweight='bold')

plt.legend(loc="upper left", fontsize=8)
plt.savefig('xg_coverage.svg')

In [23]:
xg_pickle = open('xg_ml.pickle', 'wb')
pickle.dump(mapie, xg_pickle)
xg_pickle.close()